In [1]:
import os
import shutil
from IPython import get_ipython

from PIL import Image, ImageOps
import imageio

In [2]:
notebook_path = get_ipython().starting_dir
output_folder = os.path.join(notebook_path, "diagrams")

shutil.rmtree(output_folder, ignore_errors=True)
os.makedirs(output_folder)

In [3]:
big_name = "AWS Serverless Architecture"

In [12]:
from diagrams import Diagram
from diagrams.aws.compute import Lambda
from diagrams.aws.integration import APIGateway
from diagrams.aws.storage import S3
from diagrams.aws.database import Dynamodb
from diagrams.aws.integration import SNS, SQS, Eventbridge
from diagrams.aws.security import Cognito
from diagrams.aws.management import Cloudwatch, XRay
from diagrams.aws.analytics import KinesisDataFirehose
from diagrams.aws. import StepFunctions

with Diagram("Advanced AWS Serverless Architecture", show=False):
    # Entry point
    api_gateway = APIGateway("API Gateway")
    user_auth = Cognito("User Authentication")

    # Core services
    lambda_function = Lambda("Lambda Function")
    step_functions = StepFunctions("Step Functions")

    # Storage
    s3_bucket = S3("S3 Bucket")
    dynamodb = Dynamodb("DynamoDB")
    firehose = KinesisDataFirehose("Data Firehose")

    # Messaging and event handling
    sns = SNS("SNS Topic")
    sqs = SQS("SQS Queue")
    eventbridge = Eventbridge("EventBridge")

    # Monitoring and logging
    cloudwatch = Cloudwatch("CloudWatch")
    xray = XRay("X-Ray")

    # Architecture flow
    api_gateway >> user_auth >> lambda_function
    lambda_function >> step_functions >> [dynamodb, s3_bucket, firehose]
    lambda_function >> eventbridge >> sns >> sqs
    lambda_function >> [cloudwatch, xray]



In [7]:
def find_largest_dimensions(image_folder, file_names):
    """ Find the largest width and height among all images """
    max_width, max_height = 0, 0
    for file_name in file_names:
        with Image.open(os.path.join(image_folder, file_name)) as img:
            if img.width > max_width:
                max_width = img.width
            if img.height > max_height:
                max_height = img.height
    return max_width, max_height


def pad_image_to_size(img, size):
    """ Pad image with white background to match the given size """
    padded_img = ImageOps.pad(img, size, color="white")
    return padded_img


def create_fading_gif(image_folder, output_gif, duration=5, fade_duration=2, static_duration=200, loop=0):
    images = []
    filenames = sorted([img for img in os.listdir(image_folder) if img.endswith(".png")])

    # Find the largest image size
    max_width, max_height = find_largest_dimensions(image_folder, filenames)
    largest_size = (max_width, max_height)

    for i in range(len(filenames) - 1):
        img1 = Image.open(os.path.join(image_folder, filenames[i]))
        img2 = Image.open(os.path.join(image_folder, filenames[i + 1]))

        # Pad images to the same size
        img1 = pad_image_to_size(img1, largest_size)
        img2 = pad_image_to_size(img2, largest_size)

        # Display static image for a duration
        for _ in range(static_duration):
            images.append(img1)

        # Create fading effect
        for t in range(fade_duration):
            alpha = t / fade_duration
            blended = Image.blend(img1, img2, alpha)
            images.append(blended)

    # Add and display the last image statically
    last_img = Image.open(os.path.join(image_folder, filenames[-1]))
    last_img = pad_image_to_size(last_img, largest_size)
    for _ in range(static_duration):
        images.append(last_img)

    # Save to GIF with infinite loop
    imageio.mimsave(output_gif, images, duration=duration, loop=loop)


image_folder = output_folder
output_gif = f'{big_name}.gif'  # Output GIF file name
create_fading_gif(image_folder, os.path.join(image_folder, output_gif))

IndexError: list index out of range